In [4]:
#연습문제 5-3  1
import cv2 as cv

img=cv.imread('apples.jpg')
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)

sift=cv.SIFT_create(nfeatures = 2) 
kp,des=sift.detectAndCompute(gray,None)

gray=cv.drawKeypoints(gray,kp,None,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imshow('sift', gray)

k=cv.waitKey()
cv.destroyAllWindows()

In [7]:
# 연습문제5-3 2
import cv2 as cv

img = cv.imread('apples.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

nfeatures = [4, 8, 16, 32, 64, 128, 512]

for n in nfeatures:
    sift = cv.SIFT_create(nfeatures=n)
    kp, des = sift.detectAndCompute(gray, None)

    gray = cv.drawKeypoints(gray,kp,None,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv.imshow('sift_{}'.format(n),gray)

k=cv.waitKey()
cv.destroyAllWindows()

In [9]:
#연습문제 5-6 1
import cv2 as cv
import numpy as np


img = cv.imread('apples.jpg')


octave = 0



ksize = 3
sigma = 1.6
gauss_pyr = [cv.cvtColor(img, cv.COLOR_BGR2GRAY)]
for i in range(6):
    gauss_pyr.append(cv.GaussianBlur(gauss_pyr[i], (ksize, ksize), sigma))

dog_pyr = []
for i in range(5):
    dog_pyr.append(cv.absdiff(gauss_pyr[i], gauss_pyr[i+1]))

for i in range(6):
    cv.imshow('Gaussian Image Level ' + str(i+1), gauss_pyr[i])
for i in range(5):
    cv.imshow('DoG Image Level ' + str(i+1), dog_pyr[i])

cv.waitKey()
cv.destroyAllWindows()

In [8]:
# 연습문제 5-6 2
import cv2 as cv
import numpy as np


img = cv.imread('apples.jpg')


octave = 0
n_levels = 5 


ksize = 3
sigma = 1.6
gauss_pyr = [cv.cvtColor(img, cv.COLOR_BGR2GRAY)]

for i in range(n_levels-1):
    sigma_f = np.sqrt(sigma ** 2 - (sigma / 2) ** 2)  
    ksize_f = int(2 * np.ceil(3 * sigma_f) + 1) 
    gauss_pyr.append(cv.GaussianBlur(gauss_pyr[i], (ksize_f, ksize_f), sigma_f))


dog_pyr = []
for i in range(n_levels-1):
    dog_pyr.append(cv.absdiff(gauss_pyr[i], gauss_pyr[i+1]))


for i in range(n_levels):
    cv.imshow('Gaussian Image Level ' + str(i+1), gauss_pyr[i])
for i in range(n_levels-1):
    cv.imshow('DoG Image Level ' + str(i+1), dog_pyr[i])

cv.waitKey()
cv.destroyAllWindows()

In [1]:
#연습문제 5-8
import cv2 as cv
import numpy as np

img1=cv.imread('img11.PNG')[0:350,0:580]
gray1=cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
img2=cv.imread('img2.PNG') 
gray2=cv.cvtColor(img2,cv.COLOR_BGR2GRAY)

sift=cv.SIFT_create()
kp1,des1=sift.detectAndCompute(gray1,None)
kp2,des2=sift.detectAndCompute(gray2,None)

flann_matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_match=flann_matcher.knnMatch(des1,des2,2)

T=0.7
good_match=[]  
for nearest1,nearest2 in knn_match:
    if (nearest1.distance/nearest2.distance)<T: 
        good_match.append(nearest1)

points1=np.float32([kp1[gm.queryIdx].pt for gm in good_match])
points2=np.float32([kp2[gm.trainIdx].pt for gm in good_match])

H,_=cv.findHomography(points1,points2,cv.RANSAC)

h1,w1=img1.shape[0],img1.shape[1] 
h2,w2=img2.shape[0],img2.shape[1] 

box1=np.float32([[0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0]]).reshape(4,1,2) 
box2=cv.perspectiveTransform(box1,H) 

img2=cv.polylines(img2,[np.int32(box2)],True,(0,255,0),8) 

img_match=np.empty((max(h1,h2),w1+w2,3),dtype=np.uint8)

cv.drawMatches(img1,kp1,img2,kp2,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS) 
                                                                                               
cv.imshow('Matches and Homography',img_match)

k=cv.waitKey()
cv.destroyAllWindows()